<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [ ]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# для удобства будем проставлять нумерацию в названии переменных, для соотнесения к какому заданию относится запрос
query_3_1 = f'''
    SELECT count(*)
    from vacancies v
'''

In [ ]:
# для удобства будем брать данные из полученного DF с подписью
display(f'Вакансий в базе: {int(pd.read_sql_query(query_3_1, connection).iloc[0])}')

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# общие данные можно получить по размеру таблицы (количеству записей в ней)
query_3_2 = f'''
    SELECT count(*)
    from EMPLOYERS
'''

In [ ]:
display(f'Работодателей в базе: {int(pd.read_sql_query(query_3_2, connection).iloc[0])}')

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# общие данные можно получить по размеру таблицы (количеству записей в ней)
query_3_3 = f'''
    SELECT count(*)
    from AREAS
'''

In [ ]:
display(f'Регионов в базе: {int(pd.read_sql_query(query_3_3, connection).iloc[0])}')

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
#общие данные можно получить по размеру таблицы (количеству записей в ней)
query_3_4 = f'''
    SELECT count(*)
    from INDUSTRIES
'''

In [ ]:
display(f'Сфер деятельности в базе: {int(pd.read_sql_query(query_3_4, connection).iloc[0])}')

***

Вывод:
Удалось подключиться к БД, удалось узнать размерность записей:

Вакансий в базе: 49197
Работодателей в базе: 23501
Регионов в базе: 1362
Сфер деятельности в базе: 294

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# установил LIMIT т.к. для ответа в тестировании необходимо только ТОП-5 регионов
query_4_1 = f'''
    SELECT 
    a.name, 
    count(v.id)
    from areas a
    join vacancies v on a.id = v.area_id
    group by a.id
    order by 2 desc
    limit 5
'''

In [ ]:
aw_4_1 = pd.read_sql_query(query_4_1, connection)
display(f'Вакансий в ТОП-5 регионов:')
aw_4_1

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# ввводим уловия: отсуствие пустого значения
query_4_2 = f'''
    SELECT 
   count(*)
    FROM VACANCIES v
    WHERE salary_from IS NOT NULL or salary_to IS NOT NULL
    
'''

In [ ]:
display(f'Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: {int(pd.read_sql_query(query_4_2, connection).iloc[0])}')

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# выводим среднее значение по интересуемым столбцам
query_4_3 = f'''
    SELECT 
   round(AVG(salary_from)) AVG_from, 
   round(AVG(salary_to)) AVF_To
    FROM VACANCIES
    '''

In [ ]:
aw_4_3 = pd.read_sql_query(query_4_3, connection)
display('Cредние значения для нижней и верхней границы зарплатной вилки')
aw_4_3


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# установил связку OFFSET и LIMIT для получения второго значения (вопрос теста), если нужен полный спиок - связку следует убрать
query_4_4 = f'''
    SELECT 
        schedule,
        employment, 
        count(*)
    FROM VACANCIES v
    GROUP BY 1, 2 
    ORDER BY 3 DESC
    OFFSET 1
    LIMIT 1
'''

In [ ]:
aw_4_4 = pd.read_sql_query(query_4_4, connection)
display('Вторя пара по количеству ваканчий по графику и типу:')
aw_4_4.iloc[0]

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# считаем общее количество записей с группировкой по опыту
query_4_5 = f'''
    SELECT 
        experience,
        count(*)
    FROM VACANCIES 
    GROUP BY 1
    ORDER BY 2 ASC
    '''

In [ ]:
aw_4_5 = pd.read_sql_query(query_4_5, connection)
display('Требуемый опыт работы в порядке убывания количества вакансий')
display(aw_4_5)

***

# Вывод:
- в ходе расчетов выяснилось, что самое большое количество вакансий в самых больших городах; 
- нашли средние значения верхней границы зарплатных вилок, которое составляет 110537.0;
- удалось выяснить, что самая востребованный вид работы - это удаленный при полной занятости, а также требования к стажу самые высокие - минимум 1 год;
- также есть вакансии для людей без опыта.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
#  для получения опыта подтянем через inner наименования работодателей, ограничим через LIMIT интересующий диапазон (для теста 1 и 5 место)
query_5_1 = f'''
    SELECT 
        e.name,
        count(v.id)
    FROM VACANCIES v
    JOIN EMPLOYERS e ON v.employer_id = e.id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 5
'''

In [ ]:
aw = pd.read_sql_query(query_5_1, connection)
display(f'Первое место: {aw.iloc[0][0]}')
display(f'Пятое место: {aw.iloc[4][0]}')

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# также ограничен вывод через LIMIT т.к. интересует 1я строка
query_5_2 = f'''
    SELECT 
	    a.name,
	    count(distinct v.id) vac,
	    count(distinct e.id) emp
    FROM AREAS a
    LEFT JOIN VACANCIES v ON v.area_id = a.id
    LEFT JOIN EMPLOYERS e ON e.area = a.id 
	WHERE v.id is NULL
	GROUP BY 1
	ORDER BY emp DESC
	LIMIT 1
 '''

In [ ]:
aw = pd.read_sql_query(query_5_2, connection)
display(f'Регион в котором больше всего работодателей, но нет вакансий: {aw.iloc[0][0]}')

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# также ограничен вывод через LIMIT т.к. интересует 1я строка
query_5_3 = f'''
    SELECT 
	employer_id,
	count(distinct area_id) 
    FROM VACANCIES v
	GROUP BY 1
	ORDER BY 2 DESC
	LIMIT 1 
'''

In [ ]:
aw = pd.read_sql_query(query_5_3, connection)
display(f'Максимальное количество регионов одного работодателя: {aw.iloc[0][1]}')

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# для этого используем LEFT JOIN, он оставит проспуски в строках без совпадений, они будут маркером подсчета
query_5_4 = f'''
    SELECT 
        count(*)
    FROM EMPLOYERS e
    LEFT JOIN EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
    WHERE ei.industry_id is NULL
'''

In [ ]:
aw = pd.read_sql_query(query_5_4, connection)
display(f'Количество работадателей, у которых не указана сфера деятельности. : {int(aw.iloc[0])}')

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# выводить будет только name т.к. для ответа нужно именно наименование без данных, все расчеты сделаем внутри запроса и ограничим вывод OFFSET - LIMIT. Если нужен будет полный список ограничение мождно убрать
query_5_5 = f'''
    SELECT 
		e.name
    FROM EMPLOYERS e
    LEFT JOIN EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
	GROUP BY e.id
    HAVING count(ei.industry_id) = 4
	ORDER BY e.name
	OFFSET 2 
	LIMIT 1
'''

In [ ]:
aw = pd.read_sql_query(query_5_5, connection)
display('Наименование компании : {}'.format(aw.iloc[0]['name']))

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# подтягивать наименование работодателей через JOIN не стал, т.к. нужно только количество, для этого достаточно двух таблиц INDUSTRIES и EMPLOYERS_INDUSTRIES
query_5_6 = f'''
SELECT 
   	count(ei.employer_id) cnt
FROM EMPLOYERS_INDUSTRIES ei
JOIN INDUSTRIES i on i.id = ei.industry_id
WHERE i.name = 'Разработка программного обеспечения'

'''

In [ ]:
aw = pd.read_sql_query(query_5_6, connection)
display('Количество работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».: {}'.format(int(aw.iloc[0])))

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
cities = tuple(pd.read_html(url)[0]['Город'])


In [ ]:
# через f-строку подадим кортеж полученный при парсинге, т.к. код не сильно большой, добавим Total через union ALL и аналогичный запрос. только с подсчетом записей
query_5_7 = f''' with temp AS
(SELECT a.name,
   	count(*) cnt
FROM  VACANCIES v 
left JOIN EMPLOYERS e on e.id = v.employer_id
left JOIN AREAS a on a.id = v.area_id
WHERE e.name = 'Яндекс' and a.name in {cities}
GROUP BY a.id
order by 2)
select * from temp
UNION ALL

SELECT 'Total',
   	sum(cnt)
from temp
'''

In [ ]:
aw = pd.read_sql_query(query_5_7, connection)
display('Сколько строк получилось в выборке: {}'.format(aw.shape[0]))
display('Результат получился в строке Total: {}'.format(int(aw[aw['name'] == 'Total']['cnt'])))

***

Вывод:
- в ходе исследования мы узнали о ТОП-5 работодателях по количеству вакансий, это IT и нефтегазовое направление (Яндекс - Газпром нефть) 
- выявлен регион, где заявлены работодатели, но нет вакансий, с учетом, что это оказалась страна (Россия). 
- выявлено, что есть работодатели, которые указывают вакансии в большом количестве регионов, максимум 181!
- выявлено, что 1/3 работодателей не указывает свою сферу деятельности, это не может быть связано с тем, что у работодателя несколько сфер, т.к. присутствуют работодатели, указавшие несколько сфер
- также замечено, что около 20% работодателей указали "Разработка программного обеспечения", что свидетельствует о профессиональном голоде в этой сфере

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# используем ILIKE для игнорирования регистра
query_6_1 = f'''
SELECT 
   	count(*)
FROM VACANCIES
WHERE name ILIKE '%data%' or name ILIKE '%данн%'
'''

In [ ]:
aw = pd.read_sql_query(query_6_1, connection)
display('Вакансий имеет отношение к данным: {}'.format(int(aw.iloc[0])))

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# используем ILIKE для игнорирования регистра кроме ML и HTML, исключим возможность вхожждения символов в слова
query_6_2 = f'''
SELECT 
   	count(*)
FROM VACANCIES
WHERE 	(name ILIKE '%data scientist%' or 
		name ILIKE '%data science%' or 
		name ILIKE '%исследователь данных%' or 
		(name LIKE '%ML%' and 
		name NOT LIKE '%HTML%') or
		name ILIKE '%machine learning%' or 
		name ILIKE '%машинн%обучен%')
		and
		(name ILIKE '%junior%' or
		experience ILIKE '%Нет опыта%' or
		employment ILIKE '%Стажировка%')
'''

In [ ]:
aw = pd.read_sql_query(query_6_2, connection)
display('Количество вакансий: {}'.format(int(aw.iloc[0])))

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# аналогичный предыдущему, только уберем junior-фильтр и добавим ключевые навыки
query_6_3 = f'''
SELECT 
   	count(*)
FROM VACANCIES
WHERE 	(name ILIKE '%data scientist%' or 
		name ILIKE '%data science%' or 
		name ILIKE '%исследователь данных%' or 
		(name LIKE '%ML%' and 
		name NOT LIKE '%HTML%') or
		name ILIKE '%machine learning%' or 
		name ILIKE '%машинн%обучен%')
		and
		(key_skills LIKE '%SQL%' or
		key_skills LIKE '%postgres%')
'''

In [ ]:
aw = pd.read_sql_query(query_6_3, connection)
display('Количество вакансий SQL: {}'.format(int(aw.iloc[0])))

 !!!! Примечание: на сайте курса ответ 201 не принимается. По совету ментора в пачке использовал ответ 229

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# аналогичный предыдущему, только изменим навык
query_6_4 = f'''
SELECT 
   	count(*)
FROM VACANCIES
WHERE 	(name ILIKE '%data scientist%' or 
		name ILIKE '%data science%' or 
		name ILIKE '%исследователь данных%' or 
		(name LIKE '%ML%' and 
		name NOT LIKE '%HTML%') or
		name ILIKE '%machine learning%' or 
		name ILIKE '%машинн%обучен%')
		and
		(key_skills LIKE '%Python%')
'''

In [ ]:
aw_6_4 = pd.read_sql_query(query_6_4, connection)
display('Количество вакансий для DS с Python: {}'.format(int(aw_6_4.iloc[0])))

!!!! Примечание: на сайте курса ответ 351 не принимается. По совету ментора в пачке использовал ответ 357

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# для определения количества, решил посчитать резделители. определим их количество как разницу символов с ними и без них. (добавил 1, т.к. разделителей всегда на 1 меньше, чем разделяемого)
query_6_5 = f'''
SELECT 
   	round(avg(length(key_skills)-length(replace(key_skills, CHR(9), ''))+1), 2)
FROM VACANCIES
WHERE 	(name ILIKE '%data scientist%' or 
		name ILIKE '%data science%' or 
		name ILIKE '%исследователь данных%' or 
		(name LIKE '%ML%' and 
		name NOT LIKE '%HTML%') or
		name ILIKE '%machine learning%' or 
		name ILIKE '%машинн%обучен%')
'''

In [ ]:
aw_6_5 = pd.read_sql_query(query_6_5, connection)
display('Ключевых навыков в среднем указывают в вакансиях для DS: {}'.format(float(aw_6_5.iloc[0])))

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# аналогичная схема предыдущих запросов, только исключим пропуски, для удобства дальнейшего вывода ответа для тестовой части введем наименование колонок (и по ним обратимся)
query_6_6 = f'''
SELECT 
   	experience as experience,
   	round(avg(coalesce((salary_from + salary_to) / 2, salary_from, salary_to)), 0) as avg_ds
FROM VACANCIES
WHERE 	(name ILIKE '%data scientist%' or 
		name ILIKE '%data science%' or 
		name ILIKE '%исследователь данных%' or 
		(name LIKE '%ML%' and 
		name NOT LIKE '%HTML%') or
		name ILIKE '%machine learning%' or 
		name ILIKE '%машинн%обучен%') and  
        (salary_from is not NULL or salary_to is not NULL)
GROUP BY experience
'''

In [ ]:
aw_6_6 = pd.read_sql_query(query_6_6, connection)
display('Средняя з/п для дата-сайентиста с опытом работы от 3 до 6 лет: {}'.format(int(
    aw_6_6[aw_6_6['experience'] == 'От 3 до 6 лет']['avg_ds']))
    )

***

Вывод:
В ходе исследования мы узнали, что есть спрос на дата-сайнетисов, в том числе начинающих, также можно сделать вывод, что хоть и важен Python, требования к знанию SQL имеют огромный вес в общем количестве вакансий. оценили размер заработной платы, она очень сильно зависит от опыта работы специалиста. Не маловажным фактором являются ключевые навыки, довольно большое число их указано для DS в каждой вакансии.

# Общий вывод по проекту

Общий вывод:
Перед формирование общего вывода напомню поставленную задачу и точку взгляда на датасет:

Мы представляем кадровое агентство, которое подбирает вакансии для IT-специалистов. Нам нужно оценить данные для построения модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist

С учетом этого мы видим, что очень выжным являются навыки приобретенные специалистом, опыт их работы, желаемая з/п.
Однако возможно высказать гипотезу по з/п, что это будет не критичныей показатель для junior-специалистов, т.к. уровень зарплаты растет по мере получения опыта

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes= plt.subplots(1, 2, figsize=(15, 5))
sns.lineplot(x='experience', y='avg_ds', data=aw_6_6, ax=axes[0])
sns.barplot(x='experience', y='count', data=aw_4_5, order=['Нет опыта', 'От 1 года до 3 лет', 'От 3 до 6 лет', 'Более 6 лет'], ax=axes[1])
axes[0].set_title('График зависимости средней з/п от опыта для DS')
axes[1].set_title('Количество вакансий по опыму работы')
axes[0].grid(True)

Визуализация показывает, что самые востребованные это специалисты с опытом от 1 до 3х лет, но допольно большое количество востребовано без опыта. Сделаем еще запрос для сравнения показателей для DS и остальных вакансий:

In [ ]:
query_7_1 = f'''
SELECT 
   	experience as experience,
   	round(avg(coalesce((salary_from + salary_to) / 2, salary_from, salary_to)), 0) as avg_no_DS
FROM VACANCIES
WHERE 	(name NOT ILIKE '%data scientist%' or 
		name NOT ILIKE '%data science%' or 
		name NOT ILIKE '%исследователь данных%' or 
		(name NOT LIKE '%ML%' and 
		name NOT LIKE '%HTML%') or
		name NOT ILIKE '%machine learning%' or 
		name NOT ILIKE '%машинн%обучен%') and  
        (salary_from is not NULL or salary_to is not NULL)
GROUP BY experience
'''

In [ ]:
aw_7_1 = aw_6_6.merge(pd.read_sql_query(query_7_1, connection), how='left')
aw_7_1['delta'] = aw_7_1['avg_ds'] / aw_7_1['avg_no_ds'] *100 - 100
aw_7_1

Как видим, более, чем на 80% средние зарплаты для DS выше, чем для остальных групп

Отвечая на вопрос: "что из себя представляют данные и насколько они соответствуют целям проекта"
можно с уверенностью сказать, что они состоят из описательных, количественных и качественных данных, которые возможно использовать в DS модели для целей ML

In [ ]:
# не забываем закрыть соединение после окончания работы
connection.close()